# COVID - 19 Projection Dashboard

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly
import multiprocessing
import csv
import openpyxl

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

import plotly.graph_objects as go
import plotly.express as px

from scipy.interpolate import interp1d

In [ ]:


app = dash.Dash(__name__)



In [ ]:
# loading the dataset

death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
projected_df = pd.read_csv('/home/aqi/1_Work/Data_Vis/Project_main/data_main.csv')
projected_df.head()

In [ ]:
# droping the 'Province/State' columns as it containd null values

death_df.drop('Province/State', axis=1, inplace=True)
confirmed_df.drop('Province/State', axis=1, inplace=True)
recovered_df.drop('Province/State', axis=1, inplace=True)
country_df.drop(['People_Tested', 'People_Hospitalized'], axis=1, inplace=True)

In [ ]:
# change columns name

death_df.rename(columns={'Country/Region': 'Country'}, inplace=True)
confirmed_df.rename(columns={'Country/Region': 'Country'}, inplace=True)
recovered_df.rename(columns={'Country/Region': 'Country'}, inplace=True)
country_df.rename(columns={'Country_Region': 'Country', 'Long_': 'Long'}, inplace=True)

In [ ]:
# sorting country_df with highest confirm case

country_df.sort_values('Confirmed', ascending=False, inplace=True)
projected_df.sort_values('actual_deaths', ascending=False, inplace= True)
projected_df.head()

In [ ]:
# fixing the size of circle to plot in the map

margin = projected_df['actual_deaths'].values.tolist()
circel_range = interp1d([1, max(margin)], [0.2,12])
circle_radius = circel_range(margin)

In [ ]:
# ploting world map
# fixing the size of circle

margin = projected_df['actual_deaths'].values.tolist()
circel_range = interp1d([1, max(margin)], [0.2,12])
circle_radius = circel_range(margin)



In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.SOLAR])

# navbar code
navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(html.A("Projections", href="#nav-daily-graph", style = {'color': '#fff'}), className="mr-5"),
        dbc.NavItem(html.A("Most Deaths", href="#nav-top-country-graph", style = {'color': '#fff'}), className="mr-5"),
        dbc.NavItem(html.A("Comparison", href="#nav-cr-link", style = {'color': '#fff'}), className="mr-5"),
    ],
    brand="COVID-19 Forecast",
    brand_href="/",
    color="dark",
    dark=True,
    className="p-3 fixed-top"
)

In [ ]:
# main heading

main_heading = dbc.Container(
[
    html.H1(["COVID-19 Forecast Comparision Dashboard"], className="my-5 pt-5 text-center"),
 ]
, className='pt-3')

# what is covid-19

what_is_covid = dbc.Container(
    [
        html.Div([
            html.H3('About COVID-19 Forecasting'),
            html.P("Mathematical modeling helps CDC and partners respond to the COVID-19 pandemic by informing decisions about pandemic planning, resource allocation, and implementation of social distancing measures and other interventions."),
            html.P("Forecasting teams predict numbers of deaths, hospitalizations, and cases using different types of data (e.g., COVID-19 data, demographic data, mobility data), methods, and estimates of the impacts of interventions (e.g., social distancing, use of face coverings). These forecasts are developed independently and shared publicly. It is important to bring these forecasts together to help understand how they compare with each other and how much uncertainty there is about what may happen in the near future."),
            html.Span('More information '),
            dcc.Link(' here', href='https://github.com/youyanggu/covid19-forecast-hub-evaluation/tree/master/global')
        ])
    ]
, className="mb-5")

In [ ]:
# select, country, no of days and category

world_tally = dbc.Container(
    [
        html.H2('Actual World Data & Pridiction', style = {'text-align': 'center'}),
        
        dbc.Row(
            [
                dbc.Col(children = [html.H4('Actual Deaths'), 
                        html.Div(projected_df['actual_deaths'].sum(), className='text-info', style = {'font-size': '34px', 'font-weight': '700'})],
                        width=3, className='text-center bg-light border-right p-2', style = {'border-top-left-radius': '6px', 'border-bottom-left-radius': '6px'}),
                
                dbc.Col(children = [html.H4('Baseline', style = {'padding-top': '0px'}),
                        html.Div(projected_df['Baseline'].sum(), className='text-success', style = {'font-size': '34px', 'font-weight': '700'})],
                        width=3, className='text-center bg-light border-right p-2'),
                
                dbc.Col(children = [html.H4('YYG-ParamSearch', style = {'padding-top': '0px'}), 
                        html.Div(projected_df['YYG-ParamSearch'].sum(), className='text-danger', style = {'font-size': '34px', 'font-weight': '700'})],
                        width=3, className='text-center bg-light border-right p-2'),
                
                dbc.Col(children = [html.H4('IHME-CurveFIt'),
                        html.Div(projected_df['IHME-CurveFIt'].sum(),className='text-warning', style = {'font-size': '34px', 'font-weight': '700'})],
                        width=3, className='text-center bg-light p-2', style = {'border-top-right-radius': '6px', 'border-bottom-right-radius': '6px'}),
            ]
        , className='my-4 shadow justify-content-center'),
            
    ]
)

In [ ]:
# global map heading

global_map_heading = html.H2(children='World map view', className='mt-5 py-4 pb-3 text-center')

# ploting the map
map_fig = px.scatter_mapbox(projected_df, lat="Lat", lon="Long", hover_name="Country", hover_data=["Baseline", "actual_deaths"],
                        color_discrete_sequence=["#e60039"], zoom=2, height=500, size_max=50, size=circle_radius)

map_fig.update_layout(mapbox_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0}, height=520)



In [ ]:
# daily data heading

daily_graph_heading = html.H2(id='nav-daily-graph', children='COVID-19 Projectioins and Actual Death ', className='mt-5 pb-3 text-center')


In [ ]:
# dropdown to select the country, category and number of days

daily_country = confirmed_df['Country'].unique().tolist()
daily_country_list = []

my_df_type = ['Confirmed cases', 'Death rate', 'Recovered cases']
my_df_type_list = []

for i in daily_country:
    daily_country_list.append({'label': i, 'value': i})
    
for i in my_df_type:
    my_df_type_list.append({'label': i, 'value': i})
    
# dropdown to select country
country_dropdown = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(children = [html.Label('Select Country'), 
                        html.Div(dcc.Dropdown(id = 'select-country', options = daily_country_list, value='India'))],
                        width=3, className='p-2 mr-5'),
                
                dbc.Col(children = [html.Label('Drage to choose no of Days', style = {'padding-top': '0px'}),
                        html.Div(dcc.Slider( id = 'select-date',
                                            min=10,
                                            max=len(death_df.columns[3:]),
                                            step=1,
                                            value=40
                                        ,className='p-0'), className='mt-3')],
                        width=3, className='p-2 mx-5'),
                
                dbc.Col(children = [html.Label('Select category', style = {'padding-top': '0px'}), 
                        html.Div(dcc.Dropdown(id = 'select-category', options = my_df_type_list, value='Confirmed cases'))],
                        width=3, className='p-2 ml-5'),
            ]
        , className='my-4 justify-content-center'),
            
    ]
)

In [ ]:
# create graph for daily report

def daily_graph_gen(new_df, category):
    daily_data = []
    daily_data.append(go.Scatter(
                  x=new_df['Date'], y=new_df['coronavirus'], name="Covid-19 daily report", line=dict(color='#f36')))
    
    layout = {
        'title' :'Daily ' + category +'  in ' + new_df['Country'].values[0],
        'title_font_size': 26,
        'height':450,
        'xaxis' : dict(
            title='Date',
            titlefont=dict(
            family='Courier New, monospace',
            size=24,
            color='#7f7f7f'
        )),
        'yaxis' : dict(
            title='Covid-19 cases',
            titlefont=dict(
            family='Courier New, monospace',
            size=20,
            color='#7f7f7f'
        )),
        }  
    
    figure = [{
        'data': daily_data,
        'layout': layout
    }]
    
    return figure

In [ ]:
# top 10 country with covid-19 heading

top_country_heading = html.H2(id='nav-top-country-graph', children='Top most Effected countries with COVID-19', className='mt-5 pb-3 text-center')

In [ ]:
# dropdown to select no of country
no_of_country = []

top_category = projected_df.loc[0:, ['actual_deaths', 'IHME-CurveFIt', 'YYG-ParamSearch','Baseline']].columns.tolist()
top_category_list = []

for i in no_of_country:
    no_of_country.append({'label': i, 'value': i})
    
for i in top_category:
    top_category_list.append({'label': i, 'value': i})
    

# country dropdown object
top_10_country = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(children = [html.Label('Select Projections Error Log'), 
                        html.Div(dcc.Dropdown(id = 'Error Log', options = ('error-YYG','error-IHME'), value='error-YYG'))],
                        width=3, className='p-2 mr-5'),
                
                dbc.Col(children = [html.Label('Select Source', style = {'padding-top': '0px'}), 
                        html.Div(dcc.Dropdown(id = 'Source', options = ('YYG-ParamSearch', 'YYG-ParamSearch'), value='YYG-ParamSearch'))],
                        width=3, className='p-2 ml-5'),
            ]
        , className='my-4 justify-content-center'),
            
    ]
)


In [ ]:
#confirmed and recovered cases
# heading 
cr_heading = html.H2(id='nav-cr-link', children='Actual and Forecasted deaths', className='mt-5 pb-3 text-center')

# confrirm and recovered cases
top_country = projected_df.head(10)
top_country_name = list(top_country['Country'].values)

cr = go.Figure(data=[
    go.Bar(name='Actual Deaths',marker_color='#f36', x=top_country_name, y=list(top_country['actual_deaths'])),
    go.Bar(name='Baseline', marker_color='#1abc9c',x=top_country_name, y=list(top_country['Baseline'])),
])

# Change the bar mode
cr.update_layout(barmode='group', height=600, title_text="Top 10 countires with Total and Projected Baseline Deaths")

In [ ]:
#conclusion

end = html.Div(children= [
        html.H3('Sources:'),
        html.Div([html.Span('1. The data is taken from '), dcc.Link('covid19-forecast-hub-evaluation', href='https://github.com/youyanggu/covid19-forecast-hub-evaluation/tree/master/global')]),
        html.Div([html.Span('2. Subject '), dcc.Link('Data Visualization', href='https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_data_visualization.html')]),
        html.Div([html.Span('3. To Prof.Dr.-Ing. Christoph Kinkeldey '), dcc.Link('Profile', href='https://www.mi.fu-berlin.de/en/inf/groups/hcc/members/postdocs/kinkeldey.html')]),
        html.Div([html.Span('3. Final Project ')]),
        html.H5('By: Aqib Khanzada, Anand Ravishankar, Lakshmi Sheshadri, Sharmila Ramakishnan.', style = {'margin-top': '20px', 'margin-bottom': '140px'})
])


In [ ]:
# main layout for Dash
# layout desing of all the above charts

app.layout = html.Div(
     [navbar,
     main_heading,
     what_is_covid,
     world_tally,
             
     # global map           
     html.Div(children = [global_map_heading,
         dcc.Graph(
             id='global_graph',
             figure=map_fig
         )
        ]
      ),
          
      # daily report graph
      dbc.Container([daily_graph_heading,
                    country_dropdown,
                    html.Div(id='country-total'),
         dcc.Graph(
             id='daily-graphs'
         )
        ]
      ),
    
       # top country
      dbc.Container([top_country_heading,
                    top_10_country,
         dcc.Graph(
             id='top-country-graph'
         )
        ]
      ),
        
      # confiremd and recovered cases
      dbc.Container(children = [cr_heading,
         dcc.Graph(
             id='cr',
             figure=cr
         )
        ]
      ),
      
      # conclusion
      dbc.Container(
          end
      )
    ]
)

In [ ]:
# start the server

server = app.server

In [ ]:
# call back function to make change on click
# callback for daly covid-19 case report

@app.callback(
     [Output('daily-graphs', 'figure')],
     [Input('select-country', 'value'),
      Input('select-category', 'value'),
      Input('select-date', 'value')]
)

def country_wise(country_name, df_type, number):
    # on select of category copy the dataframe to group by country
    if df_type == 'Confirmed cases':
        df_type = confirmed_df.copy(deep=True)
        category = 'COVID-19 confirmed cases'
        
    elif df_type == 'Death rate':
        df_type = death_df.copy(deep=True)
        category = 'COVID-19 Death rate'
        
    else:
        df_type = recovered_df.copy(deep=True)
        category = 'COVID-19 recovered cases'
        
    
    # group by country name
    country = df_type.groupby('Country')
    
    # select the given country
    country = country.get_group(country_name)
    
    # store daily death rate along with the date
    daily_cases = []
    case_date = []
    
    # iterate over each row
    for i, cols in enumerate(country):
        if i > 3:
            # take the sum of each column if there are multiple columns
            daily_cases.append(country[cols].sum())
            case_date.append(cols)
            zip_all_list = zip(case_date, daily_cases)
            
            # creata a data frame
            new_df = pd.DataFrame(data = zip_all_list, columns=['Date','coronavirus'])

    # append the country to the data frame
    new_df['Country'] = country['Country'].values[0]
    
    # get the daily death rate
    new_df2 = new_df.copy(deep=True)
    for i in range(len(new_df) -1):
        new_df.iloc[i+1, 1] = new_df.iloc[1+i, 1] - new_df2.iloc[i, 1]
        if new_df.iloc[i+1, 1] < 0:
            new_df.iloc[i+1, 1] = 0
    
    new_df = new_df.iloc[-number:]
    
    return (daily_graph_gen(new_df, category))


In [ ]:
# show total data for each country

@app.callback(
     [Output('country-total', 'children')],
     [Input('select-country', 'value')]
)

def total_of_country(country):
#     country = new_df['Country'].values[0]
    
    # get the country data from country_df
    my_country = country_df[country_df['Country'] == country].loc[:, ['Confirmed', 'Deaths', 'Recovered', 'Active']]
    
    country_total = dbc.Container(
    [   
        html.H4('Total case in '+ country+ ''),
        dbc.Row(
            [
                dbc.Col(children = [html.H6('Confirmed'), 
                        html.Div(my_country['Confirmed'].sum(), className='text-info', style = {'font-size': '28px', 'font-weight': '700'})],
                        width=3, className='text-center bg-light border-right pt-2', style = {'border-top-left-radius': '6px', 'border-bottom-left-radius': '6px'}),
                
                dbc.Col(children = [html.H6('Recovered', style = {'padding-top': '0px'}),
                        html.Div(my_country['Recovered'].sum(), className='text-success', style = {'font-size': '28px', 'font-weight': '700'})],
                        width=3, className='text-center bg-light border-right pt-2'),
                
                dbc.Col(children = [html.H6('Death', style = {'padding-top': '0px'}), 
                        html.Div(my_country['Deaths'].sum(), className='text-danger', style = {'font-size': '28px', 'font-weight': '700'})],
                        width=3, className='text-center bg-light border-right pt-2'),
                
                dbc.Col(children = [html.H6('Active'),
                        html.Div(my_country['Active'].sum(),className='text-warning', style = {'font-size': '28px', 'font-weight': '700'})],
                        width=3, className='text-center bg-light pt-2', style = {'border-top-right-radius': '6px', 'border-bottom-right-radius': '6px'}),
            ]
        , className='mt-1 justify-content-center'),
            
    ]
)
    
    return [country_total]

In [ ]:
# callback to show cases for top most effected countries

@app.callback(
     [Output('top-country-graph', 'figure')],
     [Input('no-of-country', 'value'),
      Input('top-category', 'value')]
    )

# method to get the top countries

def top_ten(number, sort_by):
    # sorting the columns with top death rate
    country_df2 = country_df.sort_values(by=sort_by, ascending=False)
    
    # sort country with highest number of cases
    country_df2 = country_df2.head(number)
    
    top_country_data = []
    top_country_data.append(go.Bar(x=country_df2['Country'], y=country_df2[sort_by]))
    
    layout = {
        'title': 'Top ' + str(number) +' Country - ' + sort_by + ' case',
        'title_font_size': 26,
        'height':500,
        'xaxis': dict(title = 'Countries'),
        'yaxis': dict(title = sort_by)
    }
    
    figure = [{
        'data': top_country_data,
        'layout': layout
    }]
    
    return figure

if __name__ == '__main__':
    app.run_server()

In [ ]:
if __name__ == '__main__':
    app.run_server()